In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

# Toy example 1 - Single eligible group, Single ineligible group

In [2]:
def _extract_solution_discount(m):
    """
    Get solution from optimization model
    """
    users_in = [v.x for v in m.getVars() if v.VarName.find("ineligible_val") != -1]
    users_el = [v.x for v in m.getVars() if v.VarName.find("eligible_allocation") != -1]
    edge_flows = [v.x for v in m.getVars() if v.VarName.find("edge_flows") != -1]
    
    users_in = np.reshape(users_in, (2, T))
    users_el = np.reshape(users_el, (2, T))
    edge_flows = np.reshape(edge_flows, (2, T))

    solution = {
        "users_in": users_in,
        "users_el": users_el,
        "edge_flows": edge_flows
    }
    
    return solution

In [3]:
def avg_travel_times_1(solution_set, theta):
    
    users_in = solution_set["users_in"]
    users_el = solution_set["users_el"]
    edge_flows = solution_set["edge_flows"]
    
    avg_travel_time_in = sum( users_in[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_el = sum( users_el[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_all = sum( edge_flows[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / (T*2)
    
    avg_travel_times = {
        "users_in": avg_travel_time_in,
        "users_el": avg_travel_time_el,
        "all": avg_travel_time_all
    }
    
    return avg_travel_times


In [4]:
def m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_d1 = gp.Model()

    users_in = m_d1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    edge_flows = m_d1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
    users_el = m_d1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_d1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_d1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_d1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_d1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_d1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_d1.setObjective(sum( a_1*(edge_flows[0, t]) + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*(edge_flows[1, t]) + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
                        + (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible 
        for t in range(T)), GRB.MINIMIZE)
    
    m_d1.update()

    m_d1.optimize()
    
    solution_set_d1 = _extract_solution_discount(m_d1)

    return solution_set_d1

In [5]:
def m_b1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_b1 = gp.Model()

    users_in = m_b1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    edge_flows = m_b1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
    users_el = m_b1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_b1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_b1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_b1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
    # Budget constraint satisfaction:
    if toll != 0:
        m_b1.addConstr((sum(toll*users_el[0, t] for t in range(T)) <= budget))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_b1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_b1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_b1.setObjective(sum( a_1*(edge_flows[0, t]) + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*(edge_flows[1, t]) + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
        for t in range(T)), GRB.MINIMIZE)
    
    m_b1.update()

    m_b1.optimize()
    
    solution_set_b1 = _extract_solution_discount(m_b1)

    return solution_set_b1

In [8]:
time_1 = time.time()

# Affine lane functions:
a_1 = 1.0
b_1 = 1.0
a_2 = 1.0
b_2 = 1.0
theta = np.array([[a_1, b_1], [a_2, b_2]])
T = 1
# toll = 1.0
# budget = 0.25
toll_array = (np.arange(25) + 1)/5
budget_array = (np.arange(25) + 1)/5

vot_ineligible = 1
vot_eligible = 1

solution_set_array_d1 = {}
solution_set_array_b1 = {}

# Return solution:

for toll_idx, toll in enumerate(toll_array):
    
    solution_set_array_at_toll_d1 = {}
    solution_set_array_at_toll_b1 = {}
    
    for budget_idx, budget in enumerate(budget_array):
        
        if budget > toll * T:
            continue
        
        discount_ratio = budget / (toll * T)
        
        print()
        print("Toll:", toll)
        print("Budget:", budget)
        print("Discount Ratio:", discount_ratio)
        print()
        
        solution_set_d1 = m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio)
        solution_set_b1 = m_b1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget)
        
        solution_set_array_at_toll_d1[budget] = solution_set_d1
        solution_set_array_at_toll_b1[budget] = solution_set_b1
    
    solution_set_array_d1[toll] = solution_set_array_at_toll_d1
    solution_set_array_b1[toll] = solution_set_array_at_toll_b1

time_2 = time.time()
print("Time: ", time_2 - time_1)


Toll: 0.2
Budget: 0.2
Discount Ratio: 1.0

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x68d86102
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual  

  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.02s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.57455436e+07 -9.50981300e+07  1.19e+04 2.97e+03  5.48e+06     0s
   1   2.94763846e+01 -1.70307223e+06  1.04e+01 2.59e+00  2.08e+05     0s
   2   2.89995261e+00 -3.17921145e+03  1.04e-05 2.59e-06  3.98e+02     0s
   3   2.89974226e+00 -1.11055331e+00  2.71e-09 6.75e-10  5.01e-01     0s
   4   2.78566240e+00  2.47913289e+00  1.38e-10 3.46e-11  3.83e-02     0s
   5   2.60542670e+00  2.59185612e+00  3.64e-12 1.82e-12  1.70e-03     0s
   6   2.60024154e+00  2.59998215e+00  1.82e-12 

   3   2.69908970e+00 -1.61956472e+00  4.78e-09 1.13e-09  5.40e-01     0s
   4   2.42646342e+00  2.04465527e+00  2.75e-10 6.55e-11  4.77e-02     0s
   5   2.21247106e+00  2.18502381e+00  1.46e-11 5.46e-12  3.43e-03     0s
   6   2.20113742e+00  2.19990575e+00  0.00e+00 1.82e-12  1.54e-04     0s
   7   2.20000493e+00  2.19999989e+00  0.00e+00 1.82e-12  6.28e-07     0s
   8   2.20000001e+00  2.20000001e+00  1.82e-12 1.82e-12  6.28e-10     0s

Barrier solved model in 8 iterations and 0.02 seconds (0.00 work units)
Optimal objective 2.20000001e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0xe92ea9cd
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e-01, 1e+00]
  QObjective rang

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x075668b5
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e-01, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.92731938e+07 -1.07071786e+08  1.32e+04 3.39e+03  5.80e+06     0s
   1   3.29996699e+01 -1.96057836e+06  1.17e+01 3.00e+00  2.40e+05     0s
   2   3.100003

 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.92731938e+07 -1.07071786e+08  1.32e+04 3.39e+03  5.80e+06     0s
   1   3.29996699e+01 -1.96057836e+06  1.17e+01 3.00e+00  2.40e+05     0s
   2   3.10000300e+00 -3.47124868e+03  1.17e-05 3.00e-06  4.34e+02     0s
   3   3.09995024e+00 -8.13896972e-01  1.48e-09 3.82e-10  4.89e-01     0s
   4   3.08369203e+00  2.94120423e+00  3.64e-11 9.09e-12  1.78e-02     0s
   5   3.00144277e+00  2.99624969e+00  0.00e+00 2.22e-16  6.49e-04     0s
   6   3.00022486e+00  2.99973480e+00  0.00e+00 1.82e-12  6.13e-05     0s
   7   3.00002867e+00  2.99996844e+00  0.00e+00 0.00e+00  7.53e-06     0s
   8   3.00000409e+00  2.99999557e+00  0.00e+00 1.82e-12  1.06e-06     0s
   9   3.00000058e+00  2.99999937e+00  0.00e+00 8.33e-17  1.49e-07     0s
  10   3.00000010e+00  2.99999989e+00  0.00e+00 2.31e-16  2.10e-08     0s
  11   3.00000001e+00  3.00000000e+00  1.82

...

Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.70269972e+06 -4.81593746e+07  6.51e+03 1.17e+03  4.13e+06     0s
   1   4.04198785e+00 -6.80687648e+05  5.01e+00 9.01e-01  8.37e+04     0s
   2   6.59014512e+00 -1.84186786e+03  5.01e-06 9.01e-07  2.31e+02     0s
   3   6.55724738e+00 -1.16679229e+00  1.27e-08 2.28e-09  9.65e-01     0s
   4   4.31604683e+00  3.48117666e+00  0.00e+00 0.00e+00  1.04e-01     0s
   5   4.03575708e+00  3.98602235e+00  0.00e+00 1.78e-15  6.22e-03     0s
   6   4.00383326e+00  3.99948539e+00  0.00e+00 8.88e-16  5.43e-04     0s
   7   4.00010512e+00  3.99999657e+00  1.82e-12 1.82e-12  1.36e-05     0s
   8   4.00000010e+00  4.00

   7   3.99000033e+00  3.98995631e+00  1.82e-12 0.00e+00  5.51e-06     0s
   8   3.99000000e+00  3.98999995e+00  0.00e+00 2.22e-16  5.50e-09     0s
   9   3.99000000e+00  3.99000000e+00  0.00e+00 8.88e-16  5.50e-12     0s

Barrier solved model in 9 iterations and 0.02 seconds (0.00 work units)
Optimal objective 3.99000000e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x584f0963
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 5e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 3e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objec

   6   3.64000021e+00  3.63999558e+00  0.00e+00 0.00e+00  5.78e-07     0s
   7   3.64000000e+00  3.64000002e+00  0.00e+00 1.79e-12  5.78e-10     0s

Barrier solved model in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective 3.64000000e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0xafe131c5
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 5e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 4e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+


                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.70745398e+06 -4.81591887e+07  2.17e+03 1.17e+03  4.13e+06     0s
   1   6.81180790e+00 -6.81657254e+05  1.72e+00 9.30e-01  8.38e+04     0s
   2   5.50784919e+00 -1.83276462e+03  1.72e-06 9.30e-07  2.30e+02     0s
   3   5.48150772e+00 -1.89019044e+00  4.32e-09 2.33e-09  9.21e-01     0s
   4   3.16483787e+00  2.49344791e+00  6.37e-11 3.27e-11  8.39e-02     0s
   5   3.02840495e+00  2.99301173e+00  1.82e-12 1.11e-16  4.42e-03     0s
   6   3.00943333e+00  3.00593984e+00  0.00e+00 0.00e+00  4.37e-04     0s
   7   3.00651243e+00  3.00639570e+00  0.00e+00 8.88e-16  1.46e-05     0s
   8   3.00640011e+00  3.00639999e+00  0.00e+00 8.88e-16  1.45e-08     0s
   9   3.00640000e+00  3.00639999e+00  0.00e+00 8.33e-17  1.45e-11     0s

Barrier solved model in 9 iterations and 0.02 seconds (0.00 work units)
Optimal objective 3.00640000e+00


Toll: 5.0
Budget: 4.8
Dis

   8   3.00004363e+00  2.99995309e+00  0.00e+00 9.37e-17  1.13e-05     0s
   9   3.00000617e+00  2.99999338e+00  1.82e-12 2.82e-16  1.60e-06     0s
  10   3.00000085e+00  2.99999908e+00  0.00e+00 1.82e-12  2.26e-07     0s
  11   3.00000012e+00  2.99999988e+00  0.00e+00 0.00e+00  3.19e-08     0s
  12   3.00000001e+00  2.99999999e+00  0.00e+00 1.78e-12  4.50e-09     0s

Barrier solved model in 12 iterations and 0.02 seconds (0.00 work units)
Optimal objective 3.00000001e+00

Time:  56.20250630378723


In [14]:
# Average travel times, discount minus budget:

avg_travel_time_discount_minus_budget = {}
avg_travel_time_discount_minus_budget_values = []

for toll_idx, toll in enumerate(toll_array):
    
    avg_travel_time_discount_minus_budget_at_toll = {}
    
    for budget_idx, budget in enumerate(budget_array):
        
        if budget > toll * T:
            continue
            
        diff = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_el"] - avg_travel_times_1(solution_set_array_b1[toll][budget], theta)["users_el"]
        avg_travel_time_discount_minus_budget_at_toll[budget] = diff
        avg_travel_time_discount_minus_budget_values.append(diff)
    
    avg_travel_time_discount_minus_budget[toll] = avg_travel_time_discount_minus_budget_at_toll    

# avg_travel_time_discount_minus_budget

In [16]:
print("Max average time difference, discount minus budget, value:", max(avg_travel_time_discount_minus_budget_values))
print("Min average time difference, discount minus budget, value:", min(avg_travel_time_discount_minus_budget_values))

Max average time difference, discount minus budget, value: 1.0795874773169372
Min average time difference, discount minus budget, value: -0.39999999978302236


In [9]:
edge_flows_d1 = solution_set_d1["edge_flows"]
users_el_d1 = solution_set_d1["users_el"]
users_in_d1 = solution_set_d1["users_in"]

edge_flows_b1 = solution_set_b1["edge_flows"]
users_el_b1 = solution_set_b1["users_el"]
users_in_b1 = solution_set_b1["users_in"]

print("edge_flows_d1:\n", edge_flows_d1)
print()
print("users_el_d1:\n", users_el_d1)
print()
print("users_in_d1:\n", users_in_d1)
print()

print()
print("edge_flows_b1:\n", edge_flows_b1)
print()
print("users_el_b1:\n", users_el_b1)
print()
print("users_in_b1:\n", users_in_b1)
print()

avg_travel_times_d1 = avg_travel_times_1(solution_set_d1, theta)
avg_travel_times_b1 = avg_travel_times_1(solution_set_b1, theta)

print()
print("avg_travel_times_d1:\n", avg_travel_times_d1)
print()
print("avg_travel_times_b1:\n", avg_travel_times_b1)
print()

edge_flows_d1:
 [[0.99986977]
 [1.00013023]]

users_el_d1:
 [[9.99869767e-01]
 [1.30232747e-04]]

users_in_d1:
 [[8.33335623e-11]
 [1.00000000e+00]]


edge_flows_b1:
 [[0.99986977]
 [1.00013023]]

users_el_b1:
 [[9.99869767e-01]
 [1.30232747e-04]]

users_in_b1:
 [[8.33335623e-11]
 [1.00000000e+00]]


avg_travel_times_d1:
 {'users_in': 2.000130232663629, 'users_el': 1.9998698012574647, 'all': 2.0000000169605467}

avg_travel_times_b1:
 {'users_in': 2.000130232663629, 'users_el': 1.9998698012574643, 'all': 2.0000000169605467}



# From other files:

In [ ]:
# # Solver 2, Discount:

# def OptPL_discount(toll, discount_ratio, T = T, num_eligible = num_eligible, num_ineligible = num_ineligible, 
#             vot_ineligible = vot_ineligible, alpha = bpr_true_alpha, fftt_1 = bpr_true_fftt, 
#               fftt_2 = bpr_true_fftt, c_1 = bpr_true_capacity, c_2 = bpr_true_capacity, 
#              a = const_multiplier, cap_thresh = cap_thresh_multiplier, b_1 = apx_slope, b_2 = apx_slope):
#     """
#     Function to solve convex optimization problem given a particular toll and budget value
#     """
    
#     # Initialize model
#     m1 = gp.Model()
#     # Add variables to model
# #     users_in = m1.addVars(num_ineligible, 3, T, name = 'ineligible_val', lb = 0.0, ub = 2.0)
# #     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = 0.0, ub = num_ineligible * 2.0)
# #     users_el = m1.addVars(3, T, name = 'eligible_allocation', lb = 0.0, ub = 2.0)
#     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
    
# #     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
#     users_in = m1.addVars(num_ineligible, 2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
#     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
#     users_el = m1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)
#     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = -10000.0, ub = 10000.0)   

#     ## Add constraints to Model 
    
#     # Total edge flow, express and general purpose lanes:
#     m1.addConstrs((edge_flows[j, t] == sum(users_in[i, j, t] for i in range(num_ineligible)) + \
#                   users_el[j, t] for j in range(2) for t in range(T)))
    
#     # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
#     m1.addConstrs((users_in[i, j, t]>=0 for i in range(num_ineligible) for j in range(2) for t in range(T)))
#     m1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
#     # Every eligible and ineligible user is assigned to one of three options: \
#     # (express paying, express with credit, non-express)
#     m1.addConstrs((sum(users_in[i, j, t] for j in range(2)) == 1 for i in range(num_ineligible) for t in range(T)))
#     m1.addConstrs((sum(users_el[j, t] for j in range(2)) == num_eligible for t in range(T)))
        
#     # Piecewise affine approximation:
#     m1.addConstrs((eps_flows[j, t] >= 0 for j in range(2) for t in range(T))) # Must be at least 0
#     m1.addConstrs((eps_flows[0, t] >= edge_flows[0, t] - cap_thresh*c_1 for t in range(T))) # Must be at least flow - capacity
#     m1.addConstrs((eps_flows[1, t] >= edge_flows[1, t] - cap_thresh*c_2 for t in range(T))) # Must be at least flow - capacity
    
    
#     ## To edit below:
#     # Set Objective
#     m1.setObjective(sum( a*fftt_1*(edge_flows[0, t]) + b_1*(eps_flows[0, t]**2)/2  
#                         + a*fftt_2*(edge_flows[1, t]) + b_2*(eps_flows[1, t]**2)/2 
#                         + sum( toll*users_in[i,0,t]/vot_ineligible[i,t] for i in range(num_ineligible))
#                         + sum( (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible[i,t] for i in range(num_eligible))
#         for t in range(T)), GRB.MINIMIZE)
    
#     m1.update()
    
#     return m1

# Scratch Work:

In [ ]:
# m_sample = gp.Model()
